<a href="https://colab.research.google.com/github/jmhuer/Fall2021MusicResearch/blob/main/HarmonyReductionAndDecompostion/pytorch_AEwCRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/GitYCC/crnn-pytorch
%cd crnn-pytorch/

Cloning into 'crnn-pytorch'...
remote: Enumerating objects: 41, done.
remote: Total 41 (delta 0), reused 0 (delta 0), pack-reused 41
Unpacking objects: 100% (41/41), done.
/content/crnn-pytorch


In [5]:
!pip install -r /content/crnn-pytorch/requirements.txt

     |████████████████████████████████| 20.3 MB 38.6 MB/s 
     |████████████████████████████████| 49.4 MB 142 kB/s 
     |████████████████████████████████| 2.1 MB 46.0 MB/s 
     |████████████████████████████████| 25.9 MB 9.2 kB/s 
     |████████████████████████████████| 748.9 MB 524 bytes/s 
     |████████████████████████████████| 69 kB 5.8 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.3
    Uninstalling tqdm-4.62.3:
      Successfully uninstalled tqdm-4.62.3
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu111
    Uninstalling torch-1.9.0+cu111:
      Successfully uninstalled torch-1.9.0+cu111
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: scipy
    Found ex

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
# %cd /content/crnn-pytorch/data
# !bash data/download_synth90k.sh
%cd /content/crnn-pytorch/src

/content/crnn-pytorch/src


In [14]:
from model import CRNN
from train import train_batch
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.nn import CTCLoss

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

crnn = CRNN(1, img_height=128, img_width=88, num_class=25,
            map_to_seq_hidden=64,
            rnn_hidden=256,
            leaky_relu=False)

# if reload_checkpoint:
#     crnn.load_state_dict(torch.load(reload_checkpoint, map_location=device))

crnn = crnn.to(device)
image = torch.zeros((12,1,128,88)).to(device)
fake_label = torch.ones((12, 128)).long().to(device)
print(fake_label)
fake_len = torch.ones((12)).long().to(device)
print(fake_len)

data_point = [image, fake_label, fake_len]
train_data = data_point

optimizer = optim.RMSprop(crnn.parameters(), lr=0.005)
criterion = CTCLoss(reduction='sum')
criterion.to(device)

epochs = 1
i = 1
for epoch in range(1, epochs + 1):
    print(f'epoch: {epoch}')
    tot_train_loss = 0.
    tot_train_count = 0
    for i in range(epochs):
        loss = train_batch(crnn, train_data, optimizer, criterion, device)
        train_size = train_data[0].size(0) #batch
        print(loss)
        tot_train_loss += loss
        tot_train_count += train_size

        print('train_batch_loss[', i, ']: ', loss / train_size)

        if False:
            evaluation = evaluate(crnn, valid_loader, criterion,
                                  decode_method=config['decode_method'],
                                  beam_size=config['beam_size'])
            print('valid_evaluation: loss={loss}, acc={acc}'.format(**evaluation))

            if i % save_interval == 0:
                prefix = 'crnn'
                loss = evaluation['loss']
                save_model_path = os.path.join(config['checkpoints_dir'],
                                                f'{prefix}_{i:06}_loss{loss}.pt')
                torch.save(crnn.state_dict(), save_model_path)
                print('save model at ', save_model_path)

        i += 1

    print('train_loss: ', tot_train_loss / tot_train_count)



cuda
tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
epoch: 1
~~~~~~ torch.Size([12, 1, 128, 88])
logits torch.Size([21, 12, 25])
b 12
i torch.Size([12])
t torch.Size([12])
740.8409423828125
train_batch_loss[ 0 ]:  61.73674519856771
train_loss:  61.73674519856771
